In [92]:
import numpy as np
import pandas as pd
import h5py
import scipy.sparse as sps
from datetime import timedelta
import datetime

# pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.display.max_columns = None
pd.options.display.max_rows = 100
pd.options.display.min_rows = 50
pd.options.display.precision = 2
pd.options.display.expand_frame_repr = True


In [93]:
filename = '../data/all_hourly_data.h5'

In [94]:
with h5py.File(filename, "r") as f:
    print("Keys: %s" % f.keys())


Keys: <KeysViewHDF5 ['codes', 'interventions', 'patients', 'vitals_labs', 'vitals_labs_mean']>


In [95]:
patients = pd.read_hdf(filename, key="patients")

one_hot = pd.get_dummies(patients[["gender", "admission_type", "first_careunit"]])
patients = patients.join(one_hot)
#display(patients.columns)

# include only patients with 6h < los < 600h or same for los_icu, but los_icu is in days
LOS_LOWER_THRESH = 6 #exclude if LOS is shorter than this many hours
LOS_UPPER_THRESH = 600 #exclude if LOS is longer than this many hours
patients["los"] = (patients["dischtime"] - patients["admittime"])
patients = patients[((patients["los"] > timedelta(hours=LOS_LOWER_THRESH))  & (patients["los"] < timedelta(hours=LOS_UPPER_THRESH))) | 
                    ((patients["los_icu"] * 24 > LOS_LOWER_THRESH)            & (patients["los_icu"] * 24 < LOS_UPPER_THRESH))]

MIN_AGE_THRESH = 18
patients = patients[patients["age"] > MIN_AGE_THRESH]

#fix ages set to 300+ for anonymity purposes
patients.loc[patients['age'] > 90, 'age'] = 90

# ignore following dummies, common practice to exclude 1 dummy per category
# gender_M
# admission_type_ELECTIVE
# first_careunit_CCU

patients = patients[["age", "gender_F", "admission_type_EMERGENCY", "admission_type_URGENT", "first_careunit_CSRU", "first_careunit_MICU", "first_careunit_SICU", "first_careunit_TSICU"]]
print(patients.shape)
display(patients.head())


(37543, 8)


,,,age,gender_F,admission_type_EMERGENCY,admission_type_URGENT,first_careunit_CSRU,first_careunit_MICU,first_careunit_SICU,first_careunit_TSICU
subject_id,hadm_id,icustay_id,,,,,,,,
3,145834,211552,76.53,0,1,0,0,1,0,0
4,185777,294638,47.85,1,1,0,0,1,0,0
6,107064,228232,65.94,1,0,0,0,0,1,0
9,150750,220597,41.79,0,1,0,0,1,0,0
11,194540,229441,50.15,1,1,0,0,0,1,0


In [96]:
interventions = pd.read_hdf(filename, key="interventions")
interventions["vasopressor"] = interventions[['vaso', 'vasopressin']].any(axis='columns').astype(int)
interventions = interventions[["vasopressor"]]
print(interventions.shape)
display(interventions.head())

(3183638, 1)


vasopressor
subject_id hadm_id icustay_id hours_in             
3          145834  211552     0                   0
                              1                   1
                              2                   1
                              3                   1
                              4                   1

In [97]:
# exclude patients that get vasopressor in the first 6h e.g. 0-5
excluded_subject_ids = interventions[interventions.index.get_level_values("hours_in").isin(range(6)) & interventions["vasopressor"] == 1]
excluded_subject_ids = excluded_subject_ids.index.get_level_values("subject_id").unique()
excluded_subject_ids.shape

(8406,)

In [98]:
outcome = pd.DataFrame(interventions.groupby(["subject_id", "hadm_id", "icustay_id"])["vasopressor"].agg("max"))
print(outcome.shape)
display(outcome.head())


(37543, 1)


,,,vasopressor
subject_id,hadm_id,icustay_id,
3,145834,211552,1
4,185777,294638,0
6,107064,228232,0
9,150750,220597,1
11,194540,229441,0


In [99]:
vitals_labs_mean = pd.read_hdf(filename, key="vitals_labs_mean")

# Paper name -> df name
# map (mean arterial pressure) -> pulmonary artery pressure mean
# spontaneousrr (spontaneous respiratory rate) -> respiratory rate
# urine (urine output) -> not found! :/
# alt (alanine transaminase) -> alanine aminotransferase
# ast (aspartate aminotransferase) -> asparate aminotransferase
# inr (international normalised ratio) -> prothrombin time inr
mask = vitals_labs_mean.columns.get_level_values("LEVEL2").isin(["diastolic blood pressure", "fraction inspired oxygen", "glascow coma scale total", "heart rate", 
                                                                 "pulmonary artery pressure mean", "systolic blood pressure", "respiratory rate", "oxygen saturation",
                                                                 "temperature", "urine output", "blood urea nitrogen", "magnesium", "platelets", "sodium", "alanine aminotransferase",
                                                                 "hematocrit", "partial pressure of oxygen", "asparate aminotransferase", "potassium", "white blood cell count",
                                                                 "bicarbonate", "creatinine", "lactate", "partial pressure of carbon dioxide", "glucose", "prothrombin time inr",
                                                                 "hemoglobin", "bilirubin"])

# display(vitals_labs_mean.columns[vitals_labs_mean.columns.get_level_values("LEVEL2").str.contains("weight")]) # search column names
vitals_labs_mean = vitals_labs_mean.iloc[:, mask]
vitals_labs_mean = vitals_labs_mean[~vitals_labs_mean.index.get_level_values("subject_id").isin(excluded_subject_ids)]
vitals_labs_mean_nan = vitals_labs_mean.copy()

median = vitals_labs_mean.median()
vitals_labs_mean = vitals_labs_mean.groupby('subject_id').fillna(method='ffill') # ffill by group over hours_in
vitals_labs_mean = vitals_labs_mean.fillna(median) # fill remaining by median

print(vitals_labs_mean.shape)
display(vitals_labs_mean.head(10))


(2368063, 27)


LEVEL2                                 alanine aminotransferase  \
Aggregation Function                                       mean   
subject_id hadm_id icustay_id hours_in                            
4          185777  294638     0                           40.00   
                              1                           40.00   
                              2                           40.00   
                              3                           40.00   
                              4                           40.00   
                              5                           24.00   
                              6                           24.00   
                              7                           24.00   
                              8                           24.00   
                              9                           24.00   

LEVEL2                                 asparate aminotransferase bicarbonate  \
Aggregation Function                                        mean        mean   
subject_id hadm_id icustay_id hours_in                                         
4          185777  294638     0                            52.00       25.00   
                              1                            52.00       25.00   
                              2                            52.00       25.00   
                              3                            52.00       25.00   
                              4                            52.00       25.00   
                              5                            64.00       21.00   
                              6                            64.00       21.00   
                              7                            64.00       21.00   
                              8                            64.00       21.00   
                              9                            64.00       21.00   

LEVEL2                                 bilirubin blood urea nitrogen  \
Aggregation Function                        mean                mean   
subject_id hadm_id icustay_id hours_in                                 
4          185777  294638     0             0.90               20.00   
                              1             0.90               20.00   
                              2             0.90               20.00   
                              3             0.90               20.00   
                              4             0.90               20.00   
                              5             1.52               10.00   
                              6             1.52               10.00   
                              7             1.52               10.00   
                              8             1.52               10.00   
                              9             1.52               10.00   

LEVEL2                                 creatinine diastolic blood pressure  \
Aggregation Function                         mean                     mean   
subject_id hadm_id icustay_id hours_in                                       
4          185777  294638     0              0.90                    61.00   
                              1              0.90                    63.00   
                              2              0.90                    63.00   
                              3              0.90                    57.00   
                              4              0.90                    61.00   
                              5              0.50                    61.00   
                              6              0.50                    55.00   
                              7              0.50                    58.00   
                              8              0.50                    56.00   
                              9              0.50                    60.00   

LEVEL2                                 fraction inspired oxygen  \
Aggregation Function                                       mean   
s

In [100]:
# FORMAT TO TENSOR (N x V x T)

# use only first 6 hours -> can be replaced by last 6h or random sampling, method not indicated in paper
# TODO random time between LOS_LOWER_THRESH hours_in and 90% quantile of LOS
# CONTROL_ENDTIME_UPPERQUANT = 0.9
# np.random.uniform(LOS_LOWER_THRESH,CONTROL_ENDTIME_UPPERQUANT*this_static_dat['LOS'],1)[0]
vitals_labs_mean = vitals_labs_mean[vitals_labs_mean.index.get_level_values("hours_in").isin(range(6))]
vitals_labs_mean_nan = vitals_labs_mean_nan[vitals_labs_mean_nan.index.get_level_values("hours_in").isin(range(6))]


def convert_vitals_labs_to_tensor(data):
    subject_ids = data.index.get_level_values("subject_id").unique()

    N = subject_ids.size
    V = data.shape[1]
    T = 6
    
    # fill 3d array
    tensor = np.zeros((N, V, T))
    for index, row in data.iterrows():
        # display(index)
        # display(row.shape)
        subject_id = index[0]
        hours_in = index[3]
        N_index = subject_ids.get_loc(subject_id)    # map id to index
        
        tensor[N_index, :, hours_in] = row
    return tensor


vitals_labs_mean = convert_vitals_labs_to_tensor(vitals_labs_mean)
vitals_labs_mean_nan = convert_vitals_labs_to_tensor(vitals_labs_mean_nan)
print(vitals_labs_mean.shape)
display(vitals_labs_mean[0,:,0])


(29137, 27, 6)


array([ 40.        ,  52.        ,  25.        ,   0.9       ,
        20.        ,   0.9       ,  61.        ,   0.5       ,
        14.        , 130.        ,  84.        ,  32.        ,
        10.5       ,   2.1       ,   2.        ,  94.        ,
        28.        , 111.        , 200.        ,   4.        ,
         1.3       ,  30.        ,  19.        , 139.        ,
       123.        ,  37.04999924,  10.5       ])

In [101]:
indicators = ~np.isnan(vitals_labs_mean_nan)
print(indicators.shape)
display(indicators[0,:,0])


(29137, 27, 6)


array([False, False, False, False, False, False, False, False, False,
       False, False,  True,  True,  True, False,  True,  True, False,
       False, False, False, False, False, False, False, False, False])

In [102]:
print(f"Exclude {excluded_subject_ids.size} patients...")

patients = patients[~patients.index.get_level_values("subject_id").isin(excluded_subject_ids)]
print(patients.shape)

outcome = outcome[~outcome.index.get_level_values("subject_id").isin(excluded_subject_ids)]
print(outcome.shape)

# vitals_labs_mean = vitals_labs_mean[~vitals_labs_mean.index.get_level_values("subject_id").isin(excluded_subject_ids)]
print(vitals_labs_mean.shape)

# indicators = indicators[~indicators.index.get_level_values("subject_id").isin(excluded_subject_ids)]
print(indicators.shape)

Exclude 8406 patients...
(29137, 8)
(29137, 1)
(29137, 27, 6)
(29137, 27, 6)


In [108]:
current_time = datetime.datetime.now().strftime("%Y_%m_%d__%H_%M")

OUTPUT_PATH = f"./{current_time}-vasopressor-"

print(OUTPUT_PATH)

patients.to_pickle(OUTPUT_PATH + 'patients.p')
outcome.to_pickle(OUTPUT_PATH + 'outcome.p')

np.save(OUTPUT_PATH + 'vitals_labs_mean.npy', vitals_labs_mean)
np.save(OUTPUT_PATH + 'indicators.npy', indicators)

# LOAD
# patients = pd.read_pickle("./2023_10_19__12_49-vasopressor-patients.npy")
# outcome = pd.read_pickle("./2023_10_19__12_49-vasopressor-outcome.npy")
# vitals_labs_mean = np.load("./2023_10_19__12_49-vasopressor-vitals_labs_mean.npy")
# indicators = np.load("./2023_10_19__12_49-vasopressor-indicators.npy")

print('Dumped')

./2023_10_19__12_49-vasopressor-
Dumped
